In [1]:
import pandas as pd
import numpy as np
dfherd = pd.read_pickle('results/base_od_4time_total_2lambda_odweighted.pkl')
dfxherd = dfherd[dfherd.name=='x']
dfyherd = dfherd[dfherd.name=='y']
dfzherd = dfherd[dfherd.name=='z']

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
vc_rate = np.zeros((6,26))

In [3]:
totalpop = [420697,443569,344450,526877,899111,339399,419797,308499,140361,547523,354750,479505,287613,666399,278815,193899,166374,379199,356465,195082,407864,321720,201739,411617,469439,465691]


In [4]:
for tt in range(6):
    for ii in range(26):
        new = dfyherd[dfyherd.i==ii]
        new2 = new[new.t == tt]
        newz = dfzherd[dfzherd.i==ii]
        new3 = newz[newz.t == tt]
        vc_rate[tt,ii] = (1.8/1.5)*0.139*0.255*((sum(new2.value)+sum(new3.value))/totalpop[ii])

In [5]:
vc_rate

array([[1.84406982e-02, 4.04935486e-03, 1.47254449e-03, 1.47243975e-02,
        1.57796797e-02, 5.67017236e-03, 1.84802331e-02, 2.51473956e-02,
        3.61366726e-03, 1.41691699e-02, 2.18687707e-02, 1.05779491e-03,
        1.76354320e-03, 1.16415937e-02, 1.81919176e-03, 2.61588739e-03,
        3.04866115e-03, 1.33760361e-03, 2.17635571e-02, 2.60002435e-03,
        1.24359578e-03, 2.41139699e-02, 2.51422853e-03, 1.23225705e-03,
        1.65259946e-02, 1.08917276e-03],
       [1.27711091e-02, 2.55530603e-02, 3.34579396e-03, 1.01973844e-02,
        7.83162891e-03, 0.00000000e+00, 1.27984890e-02, 1.73866044e-02,
        7.70430829e-03, 9.81286134e-03, 1.51452214e-02, 2.25520989e-03,
        3.75985931e-03, 8.06238797e-03, 3.87850157e-03, 5.57704999e-03,
        6.49972000e-03, 2.85175967e-03, 1.50723557e-02, 5.54323011e-03,
        2.65133578e-03, 1.67001345e-02, 5.36031415e-03, 1.69262002e-02,
        1.14450808e-02, 2.32210718e-03],
       [1.13221928e-02, 1.18381469e-02, 1.35525956e-02

In [6]:
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy import optimize
import pickle
num_comparts = 26
max_eval = 500
def deriv2(y, t, N, beta_I,beta_U,delta_I,delta_U,eta,theta, gamma_I,gamma_U,gamma_H,mu_I,mu_U,mu_H,nu,nuu,nuuu,e):
    S = y[0:num_comparts*1]
    E = y[num_comparts*1:num_comparts*2]
    U = y[num_comparts*2:num_comparts*3]
    I = y[num_comparts*3:num_comparts*4]
    CI = y[num_comparts*4:num_comparts*5]
    H = y[num_comparts*5:num_comparts*6]
    R = y[num_comparts*6:num_comparts*7]
    UR = y[num_comparts*7:num_comparts*8]
    D = y[num_comparts*8:num_comparts*9]
    V = y[num_comparts*9:num_comparts*10]
    vd = y[num_comparts*10:num_comparts*11]
    CR = y[num_comparts*11:num_comparts*12]
    for i in range(num_comparts):
        globals()['dS%sdt' % i] = -(S[i]*np.sum(beta_I(t)[:,i]*(I/N))+S[i]*np.sum(beta_U(t)[:,i]*(U/N))+nu(i,t)*S[i])
        globals()['dE%sdt' % i] = (S[i]*np.sum(beta_I(t)[:,i]*(I/N))+S[i]*np.sum(beta_U(t)[:,i]*(U/N)))-(delta_I(t)+delta_U(t))*E[i]+e(t)*(V[i]*np.sum(beta_I(t)[:,i]*(I/N))+V[i]*np.sum(beta_U(t)[:,i]*(U/N)))
        globals()['dU%sdt' % i] = (delta_U(t))*E[i] - (gamma_U(t)+mu_U(t)+theta(t))*U[i]
        globals()['dI%sdt' % i] = (delta_I(t))*E[i] - (gamma_I(t)+mu_I(t)+eta(t))*I[i]
        globals()['dCI%sdt' % i] = (delta_I(t))*E[i]+theta(t)*U[i]
        globals()['dH%sdt' % i] = eta(t)*I[i]+theta(t)*U[i] - (gamma_H(t)+mu_H(t))*H[i]
        globals()['dR%sdt' % i] = gamma_H(t)*H[i]+gamma_I(t)*I[i]-nu(i,t)*R[i]
        globals()['dUR%sdt' % i] = gamma_U(t)*U[i]-nu(i,t)*UR[i]
        globals()['dD%sdt' % i] = mu_U(t)*U[i]+mu_I(t)*I[i]+mu_H(t)*H[i]
        globals()['dV%sdt' % i] = nu(i,t)*S[i]-e(t)*(V[i]*np.sum(beta_I(t)[:,i]*(I/N))+V[i]*np.sum(beta_U(t)[:,i]*(U/N)))+nu(i,t)*R[i]+nu(i,t)*UR[i]
        globals()['dVd%sdt' % i] = nu(i,t)*S[i]+nu(i,t)*R[i]+nu(i,t)*UR[i]
        globals()['dCR%sdt' % i] = gamma_H(t)*H[i]+gamma_I(t)*I[i]
    return dS0dt,dS1dt,dS2dt,dS3dt,dS4dt,dS5dt,dS6dt,dS7dt, dS8dt,dS9dt,dS10dt,dS11dt,dS12dt,dS13dt,dS14dt,dS15dt, dS16dt,dS17dt,dS18dt,dS19dt,dS20dt,dS21dt,dS22dt,dS23dt, dS24dt,dS25dt,dE0dt,dE1dt,dE2dt,dE3dt,dE4dt,dE5dt,dE6dt,dE7dt, dE8dt,dE9dt,dE10dt,dE11dt,dE12dt,dE13dt,dE14dt,dE15dt, dE16dt,dE17dt,dE18dt,dE19dt,dE20dt,dE21dt,dE22dt,dE23dt, dE24dt,dE25dt,dU0dt,dU1dt,dU2dt,dU3dt,dU4dt,dU5dt,dU6dt,dU7dt, dU8dt,dU9dt,dU10dt,dU11dt,dU12dt,dU13dt,dU14dt,dU15dt, dU16dt,dU17dt,dU18dt,dU19dt,dU20dt,dU21dt,dU22dt,dU23dt, dU24dt,dU25dt, dI0dt,dI1dt,dI2dt,dI3dt,dI4dt,dI5dt,dI6dt,dI7dt, dI8dt,dI9dt,dI10dt,dI11dt,dI12dt,dI13dt,dI14dt,dI15dt, dI16dt,dI17dt,dI18dt,dI19dt,dI20dt,dI21dt,dI22dt,dI23dt, dI24dt,dI25dt,dCI0dt,dCI1dt,dCI2dt,dCI3dt,dCI4dt,dCI5dt,dCI6dt,dCI7dt, dCI8dt,dCI9dt,dCI10dt,dCI11dt,dCI12dt,dCI13dt,dCI14dt,dCI15dt, dCI16dt,dCI17dt,dCI18dt,dCI19dt,dCI20dt,dCI21dt,dCI22dt,dCI23dt, dCI24dt,dCI25dt, dH0dt,dH1dt,dH2dt,dH3dt,dH4dt,dH5dt,dH6dt,dH7dt, dH8dt,dH9dt,dH10dt,dH11dt,dH12dt,dH13dt,dH14dt,dH15dt, dH16dt,dH17dt,dH18dt,dH19dt,dH20dt,dH21dt,dH22dt,dH23dt, dH24dt,dH25dt,dR0dt,dR1dt,dR2dt,dR3dt,dR4dt,dR5dt,dR6dt,dR7dt, dR8dt,dR9dt,dR10dt,dR11dt,dR12dt,dR13dt,dR14dt,dR15dt, dR16dt,dR17dt,dR18dt,dR19dt,dR20dt,dR21dt,dR22dt,dR23dt, dR24dt,dR25dt, dUR0dt,dUR1dt,dUR2dt,dUR3dt,dUR4dt,dUR5dt,dUR6dt,dUR7dt, dUR8dt,dUR9dt,dUR10dt,dUR11dt,dUR12dt,dUR13dt,dUR14dt,dUR15dt, dUR16dt,dUR17dt,dUR18dt,dUR19dt,dUR20dt,dUR21dt,dUR22dt,dUR23dt, dUR24dt,dUR25dt,dD0dt,dD1dt,dD2dt,dD3dt,dD4dt,dD5dt,dD6dt,dD7dt, dD8dt,dD9dt,dD10dt,dD11dt,dD12dt,dD13dt,dD14dt,dD15dt, dD16dt,dD17dt,dD18dt,dD19dt,dD20dt,dD21dt,dD22dt,dD23dt, dD24dt,dD25dt ,dV0dt,dV1dt,dV2dt,dV3dt,dV4dt,dV5dt,dV6dt,dV7dt, dV8dt,dV9dt,dV10dt,dV11dt,dV12dt,dV13dt,dV14dt,dV15dt, dV16dt,dV17dt,dV18dt,dV19dt,dV20dt,dV21dt,dV22dt,dV23dt, dV24dt,dV25dt ,dVd0dt,dVd1dt,dVd2dt,dVd3dt,dVd4dt,dVd5dt,dVd6dt,dVd7dt, dVd8dt,dVd9dt,dVd10dt,dVd11dt,dVd12dt,dVd13dt,dVd14dt,dVd15dt, dVd16dt,dVd17dt,dVd18dt,dVd19dt,dVd20dt,dVd21dt,dVd22dt,dVd23dt, dVd24dt,dVd25dt, dCR0dt,dCR1dt,dCR2dt,dCR3dt,dCR4dt,dCR5dt,dCR6dt,dCR7dt, dCR8dt,dCR9dt,dCR10dt,dCR11dt,dCR12dt,dCR13dt,dCR14dt,dCR15dt, dCR16dt,dCR17dt,dCR18dt,dCR19dt,dCR20dt,dCR21dt,dCR22dt,dCR23dt, dCR24dt,dCR25dt
def Model(s2,s_u2,gi1,gu1,gh1,eta_m,mi1,mu1,mh1,theta_eta,di1,du1,nu1,nuu1,nuuu1,eff1):
    #N0 = np.array([396357,2228821,1799204,1167286,666081,1048734,1322943,1568950]) # need to change to num_comparts
    S0 = saved[0:num_comparts] #Susceptible
    E0 = saved[num_comparts*1:num_comparts*2] #exposed
    U0 = saved[num_comparts*2:num_comparts*3] #unidentified infectious
    I0 = saved[num_comparts*3:num_comparts*4] #identified infectious
    CI0 = saved[num_comparts*4:num_comparts*5] #cumulative infectious
    H0 = saved[num_comparts*5:num_comparts*6] #hospitalized
    R0 = saved[num_comparts*6:num_comparts*7] #recovered from hospitalized and identified infectious
    UR0 = saved[num_comparts*7:num_comparts*8] #recovered from unidentified infectious
    D0 = saved[num_comparts*8:num_comparts*9] #death
    V0 = saved[num_comparts*9:num_comparts*10] #vaccinated
    Vd0 = saved[num_comparts*10:num_comparts*11] #cumulative vaccinated
    CR0 = saved[num_comparts*11:num_comparts*12] #cumulative recovering
    N0 =I0 + R0 +UR0+ H0 + U0 + E0 +S0 + V0
    y0 = np.append(np.append(np.append(np.append(np.append(np.append(np.append(np.append(np.append(np.append(np.append(S0,E0),U0),I0),CI0),H0),R0),UR0),D0),V0),Vd0),CR0)
    # Incubation rate, delta_I, delta_U
    def deltai(t):
        return di1#*0.93+0.07
    def deltau(t):
        return du1#*4.43+0.07
    #rate of hospitalization from identified infected
    def eta(t):
        return eta_m
    def theta(t):
        return eta_m*theta_eta
    #rate of recovery, gamma_I, gamma_U, gamma_H, rate of death, mu_I, mu_U, mu_H
    def mu_I(t):
        return mi1*0.048+0.002#*np.exp(-mi2*t)
    def mu_U(t):
        return mu1*0.018+0.002#*np.exp(-mu2*t)
    def mu_H(t):
        return mh1*0.048+0.002#*np.exp(-mh2*t)
    def gamma_I(t):
        return gi1#*0.195+.005#*np.exp(gi2*t)
    def gamma_U(t):
        return gu1#*0.195+.005#*np.exp(gu2*t)
    def gamma_H(t):
        return gh1#*0.195+.005#*np.exp(gh2*t)
    # transmission rate 
    def beta_I(t):
        xx = flowin.copy()
        return (s2)*xx
    def beta_U(t):
        xx = flowin.copy()
        return (s_u2)*xx
    #vaccination rate nu- from S, nuu- form R, nuuu- from UR
    def nu(i,t):
        if int_ ==1:
            tt = t
        elif int_ ==2:
            tt = t +45
        elif int_ ==3:
            tt = t +45+45
        elif int_ ==4:
            tt = t +45+45+45 
        elif int_ ==5:
            tt = t +45+45+45+75
        if tt<120:
            return 0
        elif tt<150:
            return vc_rate[0,i]
        elif tt<180:
            return vc_rate[1,i]
        elif tt<210:
            return vc_rate[2,i]
        elif tt<240:
            return vc_rate[3,i]
        elif tt<270:
            return vc_rate[4,i]
        else:
            return vc_rate[5,i]
    def nuu(t):
        print(t)
        if post == 1:
            return nuu1
        else:
            return 0
    def nuuu(t):
        if post == 1:
            return nuuu1
        else:
            return 0
    # rate from vaccinated to exposed
    def eff(t):
        return 0.001
    ret = odeint(deriv2, y0, t, args=(N0,beta_I,beta_U,deltai,deltau,eta,theta, gamma_I,gamma_U,gamma_H,mu_I,mu_U,mu_H,nu,nuu,nuuu,eff))
    dS0dt,dS1dt,dS2dt,dS3dt,dS4dt,dS5dt,dS6dt,dS7dt, dS8dt,dS9dt,dS10dt,dS11dt,dS12dt,dS13dt,dS14dt,dS15dt, dS16dt,dS17dt,dS18dt,dS19dt,dS20dt,dS21dt,dS22dt,dS23dt, dS24dt,dS25dt,dE0dt,dE1dt,dE2dt,dE3dt,dE4dt,dE5dt,dE6dt,dE7dt, dE8dt,dE9dt,dE10dt,dE11dt,dE12dt,dE13dt,dE14dt,dE15dt, dE16dt,dE17dt,dE18dt,dE19dt,dE20dt,dE21dt,dE22dt,dE23dt, dE24dt,dE25dt,dU0dt,dU1dt,dU2dt,dU3dt,dU4dt,dU5dt,dU6dt,dU7dt, dU8dt,dU9dt,dU10dt,dU11dt,dU12dt,dU13dt,dU14dt,dU15dt, dU16dt,dU17dt,dU18dt,dU19dt,dU20dt,dU21dt,dU22dt,dU23dt, dU24dt,dU25dt, dI0dt,dI1dt,dI2dt,dI3dt,dI4dt,dI5dt,dI6dt,dI7dt, dI8dt,dI9dt,dI10dt,dI11dt,dI12dt,dI13dt,dI14dt,dI15dt, dI16dt,dI17dt,dI18dt,dI19dt,dI20dt,dI21dt,dI22dt,dI23dt, dI24dt,dI25dt,dCI0dt,dCI1dt,dCI2dt,dCI3dt,dCI4dt,dCI5dt,dCI6dt,dCI7dt, dCI8dt,dCI9dt,dCI10dt,dCI11dt,dCI12dt,dCI13dt,dCI14dt,dCI15dt, dCI16dt,dCI17dt,dCI18dt,dCI19dt,dCI20dt,dCI21dt,dCI22dt,dCI23dt, dCI24dt,dCI25dt, dH0dt,dH1dt,dH2dt,dH3dt,dH4dt,dH5dt,dH6dt,dH7dt, dH8dt,dH9dt,dH10dt,dH11dt,dH12dt,dH13dt,dH14dt,dH15dt, dH16dt,dH17dt,dH18dt,dH19dt,dH20dt,dH21dt,dH22dt,dH23dt, dH24dt,dH25dt,dR0dt,dR1dt,dR2dt,dR3dt,dR4dt,dR5dt,dR6dt,dR7dt, dR8dt,dR9dt,dR10dt,dR11dt,dR12dt,dR13dt,dR14dt,dR15dt, dR16dt,dR17dt,dR18dt,dR19dt,dR20dt,dR21dt,dR22dt,dR23dt, dR24dt,dR25dt, dUR0dt,dUR1dt,dUR2dt,dUR3dt,dUR4dt,dUR5dt,dUR6dt,dUR7dt, dUR8dt,dUR9dt,dUR10dt,dUR11dt,dUR12dt,dUR13dt,dUR14dt,dUR15dt, dUR16dt,dUR17dt,dUR18dt,dUR19dt,dUR20dt,dUR21dt,dUR22dt,dUR23dt, dUR24dt,dUR25dt,dD0dt,dD1dt,dD2dt,dD3dt,dD4dt,dD5dt,dD6dt,dD7dt, dD8dt,dD9dt,dD10dt,dD11dt,dD12dt,dD13dt,dD14dt,dD15dt, dD16dt,dD17dt,dD18dt,dD19dt,dD20dt,dD21dt,dD22dt,dD23dt, dD24dt,dD25dt ,dV0dt,dV1dt,dV2dt,dV3dt,dV4dt,dV5dt,dV6dt,dV7dt, dV8dt,dV9dt,dV10dt,dV11dt,dV12dt,dV13dt,dV14dt,dV15dt, dV16dt,dV17dt,dV18dt,dV19dt,dV20dt,dV21dt,dV22dt,dV23dt, dV24dt,dV25dt ,dVd0dt,dVd1dt,dVd2dt,dVd3dt,dVd4dt,dVd5dt,dVd6dt,dVd7dt, dVd8dt,dVd9dt,dVd10dt,dVd11dt,dVd12dt,dVd13dt,dVd14dt,dVd15dt, dVd16dt,dVd17dt,dVd18dt,dVd19dt,dVd20dt,dVd21dt,dVd22dt,dVd23dt, dVd24dt,dVd25dt, dCR0dt,dCR1dt,dCR2dt,dCR3dt,dCR4dt,dCR5dt,dCR6dt,dCR7dt, dCR8dt,dCR9dt,dCR10dt,dCR11dt,dCR12dt,dCR13dt,dCR14dt,dCR15dt, dCR16dt,dCR17dt,dCR18dt,dCR19dt,dCR20dt,dCR21dt,dCR22dt,dCR23dt, dCR24dt,dCR25dt = ret.T
    return ret.T
def cost_fucntion5(x):
    [s2,s_u2,gi1,gu1,gh1,eta_m,mi1,mu1,mh1,theta_eta,di1,du1,nu1,nuu1,nuuu1,eff1] = x
    dS0dt,dS1dt,dS2dt,dS3dt,dS4dt,dS5dt,dS6dt,dS7dt, dS8dt,dS9dt,dS10dt,dS11dt,dS12dt,dS13dt,dS14dt,dS15dt, dS16dt,dS17dt,dS18dt,dS19dt,dS20dt,dS21dt,dS22dt,dS23dt, dS24dt,dS25dt,dE0dt,dE1dt,dE2dt,dE3dt,dE4dt,dE5dt,dE6dt,dE7dt, dE8dt,dE9dt,dE10dt,dE11dt,dE12dt,dE13dt,dE14dt,dE15dt, dE16dt,dE17dt,dE18dt,dE19dt,dE20dt,dE21dt,dE22dt,dE23dt, dE24dt,dE25dt,dU0dt,dU1dt,dU2dt,dU3dt,dU4dt,dU5dt,dU6dt,dU7dt, dU8dt,dU9dt,dU10dt,dU11dt,dU12dt,dU13dt,dU14dt,dU15dt, dU16dt,dU17dt,dU18dt,dU19dt,dU20dt,dU21dt,dU22dt,dU23dt, dU24dt,dU25dt, dI0dt,dI1dt,dI2dt,dI3dt,dI4dt,dI5dt,dI6dt,dI7dt, dI8dt,dI9dt,dI10dt,dI11dt,dI12dt,dI13dt,dI14dt,dI15dt, dI16dt,dI17dt,dI18dt,dI19dt,dI20dt,dI21dt,dI22dt,dI23dt, dI24dt,dI25dt,dCI0dt,dCI1dt,dCI2dt,dCI3dt,dCI4dt,dCI5dt,dCI6dt,dCI7dt, dCI8dt,dCI9dt,dCI10dt,dCI11dt,dCI12dt,dCI13dt,dCI14dt,dCI15dt, dCI16dt,dCI17dt,dCI18dt,dCI19dt,dCI20dt,dCI21dt,dCI22dt,dCI23dt, dCI24dt,dCI25dt, dH0dt,dH1dt,dH2dt,dH3dt,dH4dt,dH5dt,dH6dt,dH7dt, dH8dt,dH9dt,dH10dt,dH11dt,dH12dt,dH13dt,dH14dt,dH15dt, dH16dt,dH17dt,dH18dt,dH19dt,dH20dt,dH21dt,dH22dt,dH23dt, dH24dt,dH25dt,dR0dt,dR1dt,dR2dt,dR3dt,dR4dt,dR5dt,dR6dt,dR7dt, dR8dt,dR9dt,dR10dt,dR11dt,dR12dt,dR13dt,dR14dt,dR15dt, dR16dt,dR17dt,dR18dt,dR19dt,dR20dt,dR21dt,dR22dt,dR23dt, dR24dt,dR25dt, dUR0dt,dUR1dt,dUR2dt,dUR3dt,dUR4dt,dUR5dt,dUR6dt,dUR7dt, dUR8dt,dUR9dt,dUR10dt,dUR11dt,dUR12dt,dUR13dt,dUR14dt,dUR15dt, dUR16dt,dUR17dt,dUR18dt,dUR19dt,dUR20dt,dUR21dt,dUR22dt,dUR23dt, dUR24dt,dUR25dt,dD0dt,dD1dt,dD2dt,dD3dt,dD4dt,dD5dt,dD6dt,dD7dt, dD8dt,dD9dt,dD10dt,dD11dt,dD12dt,dD13dt,dD14dt,dD15dt, dD16dt,dD17dt,dD18dt,dD19dt,dD20dt,dD21dt,dD22dt,dD23dt, dD24dt,dD25dt ,dV0dt,dV1dt,dV2dt,dV3dt,dV4dt,dV5dt,dV6dt,dV7dt, dV8dt,dV9dt,dV10dt,dV11dt,dV12dt,dV13dt,dV14dt,dV15dt, dV16dt,dV17dt,dV18dt,dV19dt,dV20dt,dV21dt,dV22dt,dV23dt, dV24dt,dV25dt ,dVd0dt,dVd1dt,dVd2dt,dVd3dt,dVd4dt,dVd5dt,dVd6dt,dVd7dt, dVd8dt,dVd9dt,dVd10dt,dVd11dt,dVd12dt,dVd13dt,dVd14dt,dVd15dt, dVd16dt,dVd17dt,dVd18dt,dVd19dt,dVd20dt,dVd21dt,dVd22dt,dVd23dt, dVd24dt,dVd25dt, dCR0dt,dCR1dt,dCR2dt,dCR3dt,dCR4dt,dCR5dt,dCR6dt,dCR7dt, dCR8dt,dCR9dt,dCR10dt,dCR11dt,dCR12dt,dCR13dt,dCR14dt,dCR15dt, dCR16dt,dCR17dt,dCR18dt,dCR19dt,dCR20dt,dCR21dt,dCR22dt,dCR23dt, dCR24dt,dCR25dt = Model(s2,s_u2,gi1,gu1,gh1,eta_m,mi1,mu1,mh1,theta_eta,di1,du1,nu1,nuu1,nuuu1,eff1)#,dii2,duu2,sii2,suu2,muii2,muuu2,muhh2,gii2,guu2,ghh2
    deathes = np.array((dD0dt,dD1dt,dD2dt,dD3dt,dD4dt,dD5dt,dD6dt,dD7dt, dD8dt,dD9dt,dD10dt,dD11dt,dD12dt,dD13dt,dD14dt,dD15dt, dD16dt,dD17dt,dD18dt,dD19dt,dD20dt,dD21dt,dD22dt,dD23dt, dD24dt,dD25dt))    
    deathes = deathes.T
    vaacines = np.array((dVd0dt,dVd1dt,dVd2dt,dVd3dt,dVd4dt,dVd5dt,dVd6dt,dVd7dt,dVd8dt,dVd9dt,dVd10dt,dVd11dt,dVd12dt,dVd13dt,dVd14dt,dVd15dt, dVd16dt,dVd17dt,dVd18dt,dVd19dt,dVd20dt,dVd21dt,dVd22dt,dVd23dt, dVd24dt,dVd25dt))    
    vaacines = vaacines.T
    cases = np.array((dI0dt+dH0dt+dR0dt+dD0dt, dI1dt+dH1dt+dR1dt+dD1dt, dI2dt+dH2dt+dR2dt+dD2dt, dI3dt+dH3dt+dR3dt+dD3dt, dI4dt+dH4dt+dR4dt+dD4dt, dI5dt+dH5dt+dR5dt+dD5dt, dI6dt+dH6dt+dR6dt+dD6dt, dI7dt+dH7dt+dR7dt+dD7dt, dI8dt+dH8dt+dR8dt+dD8dt, dI9dt+dH9dt+dR9dt+dD9dt, dI10dt+dH10dt+dR10dt+dD10dt, dI11dt+dH11dt+dR11dt+dD11dt, dI12dt+dH12dt+dR12dt+dD12dt, dI13dt+dH13dt+dR13dt+dD13dt, dI14dt+dH14dt+dR14dt+dD14dt, dI15dt+dH15dt+dR15dt+dD15dt, dI16dt+dH16dt+dR16dt+dD16dt, dI17dt+dH17dt+dR17dt+dD17dt, dI18dt+dH18dt+dR18dt+dD18dt, dI19dt+dH19dt+dR19dt+dD19dt, dI20dt+dH20dt+dR20dt+dD20dt, dI21dt+dH21dt+dR21dt+dD21dt, dI22dt+dH22dt+dR22dt+dD22dt, dI23dt+dH23dt+dR23dt+dD23dt, dI24dt+dH24dt+dR24dt+dD24dt, dI25dt+dH25dt+dR25dt+dD25dt))    
    cases = cases.T
    hospitalized = np.array((dH0dt,dH1dt,dH2dt,dH3dt,dH4dt,dH5dt,dH6dt,dH7dt, dH8dt,dH9dt,dH10dt,dH11dt,dH12dt,dH13dt,dH14dt,dH15dt, dH16dt,dH17dt,dH18dt,dH19dt,dH20dt,dH21dt,dH22dt,dH23dt, dH24dt,dH25dt))    
    hospitalized = hospitalized.T
    #unidentified = np.array((dU0dt,dU1dt,dU2dt,dU3dt,dU4dt,dU5dt,dU6dt,dU7dt, dU8dt,dU9dt,dU10dt,dU11dt,dU12dt,dU13dt,dU14dt,dU15dt, dU16dt,dU17dt,dU18dt,dU19dt,dU20dt,dU21dt,dU22dt,dU23dt, dU24dt,dU25dt)) 
    #unidentified = unidentified.T
    identified = np.array((dI0dt,dI1dt,dI2dt,dI3dt,dI4dt,dI5dt,dI6dt,dI7dt, dI8dt,dI9dt,dI10dt,dI11dt,dI12dt,dI13dt,dI14dt,dI15dt, dI16dt,dI17dt,dI18dt,dI19dt,dI20dt,dI21dt,dI22dt,dI23dt, dI24dt,dI25dt)) 
    identified = identified.T
    #exposed = np.array((dE0dt,dE1dt,dE2dt,dE3dt,dE4dt,dE5dt,dE6dt,dE7dt, dE8dt,dE9dt,dE10dt,dE11dt,dE12dt,dE13dt,dE14dt,dE15dt, dE16dt,dE17dt,dE18dt,dE19dt,dE20dt,dE21dt,dE22dt,dE23dt, dE24dt,dE25dt)) 
    #exposed = exposed.T
    #max_underreport = np.max(unidentified/identified)
    #max_ratio = np.max(exposed/(unidentified+identified))
    res = 0
    #if max_underreport > 10:
    #    res+=999999999999
    #if max_ratio > 10:
    #    res+=999999999999

    case_rate_score = np.sum(abs(identified-w_case_daily),axis=1)
    vac_score = np.sum(abs(np.sum(vaacines,axis=1)-np.sum(actual_v,axis=1)))
    case_score = np.sum(abs(cases-actual_case),axis=1)
    death_score = np.sum(abs(deathes-actual_death),axis=1)
    hos_score = np.sum(abs(np.sum(hospitalized,axis=1)-actual_hos))
    res += np.sum(case_score)+10*np.sum(death_score)+0.5*np.sum(vac_score)+5*hos_score+2*np.sum(case_rate_score)
    print(res)
    return res

In [7]:
results = pickle.load( open( "data/resultscaserateincluded_post.p", "rb" ) )
numbers = pickle.load( open( "data/numberscaserateincluded_post.p", "rb" ) )

/var/folders/b3/kkdtnh4d4gjgrk45d_yrf4100000gn/T/ipykernel_74190/2444286603.py:1: DeprecationWarning: Please use `OptimizeResult` from the `scipy.optimize` namespace, the `scipy.optimize.optimize` namespace is deprecated.
  results = pickle.load( open( "data/resultscaserateincluded_post.p", "rb" ) )


In [8]:
flowin = np.ones((num_comparts,num_comparts))
ffdata = pd.read_csv('data/region_flow_03_01_2020_03_18_2020_Weekday.csv')
ff=np.zeros((8,8))
ff[:,0]=ffdata['1.0'].values[0:8]
ff[:,1]=ffdata['2.0'].values[0:8]
ff[:,2]=ffdata['3.0'].values[0:8]
ff[:,3]=ffdata['4.0'].values[0:8]
ff[:,4]=ffdata['5.0'].values[0:8]
ff[:,5]=ffdata['6.0'].values[0:8]
ff[:,6]=ffdata['7.0'].values[0:8]
ff[:,7]=ffdata['8.0'].values[0:8]
list_of_health_dist={}
list_of_health_dist[0]=1
list_of_health_dist[1]=4+list_of_health_dist[0]
list_of_health_dist[2]=5+list_of_health_dist[1]
list_of_health_dist[3]=3+list_of_health_dist[2]
list_of_health_dist[4]=1+list_of_health_dist[3]
list_of_health_dist[5]=4+list_of_health_dist[4]
list_of_health_dist[6]=4+list_of_health_dist[5]
list_of_health_dist[7]=4+list_of_health_dist[6]

for j in range(num_comparts):
    for k in range(num_comparts):
        row = np.min(np.where([list_of_health_dist[ii]>j for ii in range(8)]))
        col = np.min(np.where([list_of_health_dist[ii]>k for ii in range(8)]))
        if row ==0:
            size_r = 1
        else:
            size_r = list_of_health_dist[row]-list_of_health_dist[row-1]
        if col ==0:
            size_c = 1
        else:
            size_c = list_of_health_dist[col]-list_of_health_dist[col-1]
        value = ff[row,col]/(size_c*size_r)
        flowin[j,k] = value


flowin = flowin/100
#flowin = np.ones((num_comparts,num_comparts))

In [9]:
post = 0
int_=1
T2 = 46
T = 0
dt = 1/24
d = np.genfromtxt ('data/death.csv', delimiter=",")
c = np.genfromtxt ('data/cases.csv', delimiter=",")
vv= np.genfromtxt ('data/vac.csv', delimiter=",")
h = np.genfromtxt ('data/hospitalization_data.csv',delimiter=",")
actual_case = np.zeros((T2,num_comparts))
case_daily = np.zeros((T2,num_comparts))
actual_death = np.zeros((T2,num_comparts))
actual_v = np.zeros((T2,num_comparts))
actual_hos = np.zeros((T2,num_comparts))
count = 0 
for i in range(0,T2):
    actual_case[i,:] = np.round(c[i,:],0)
    case_daily[i,:] = np.round(c[i,:],0)
    actual_death[i,:] = np.round(d[i,:],0)
    actual_hos[i] = np.round(h[i,:],0)
    if i>=300:   
        actual_v[i,:] = np.round(vv[i-300,:],0)     
for i in range(0,T2):
    if i>0:
        actual_case[i,:] += actual_case[i-1,:]
        actual_death[i,:] += actual_death[i-1,:]
w_case_daily = case_daily.copy()
for i in range(case_daily.shape[1]):
    for t in range(6,case_daily.shape[0]):
        w_case_daily[t,i] = np.mean(case_daily[t-6:t+1,i])
t = np.linspace(0, T2-T-1, T2-T)

In [10]:
#pop over 60 by health districts
pop = np.array([55415,72792,75398,82535,160141,78341,73026,62638,28462,102541,53179,82901,49185,134649,32022,18640,14717,58048,63922,31163,52992,57734,41987,63434,72261,94321])
#proportion over 60 by health districts
per = np.array([0.141,0.161,0.220,0.161,0.1820,0.225,0.166,0.203,0.199,0.1890,0.154,0.168,0.158,0.2060,0.113,0.096,0.085,0.152,0.178,0.152,0.124,0.179,0.204,0.152,0.153,0.205])
totalpop = (pop/per)
totalpop = np.array([420697,443569,344450,526877,899111,339399,419797,308499,140361,547523,354750,479505,287613,666399,278815,193899,166374,379199,356465,195082,407864,321720,201739,411617,469439,465691])

N0=np.transpose(totalpop)
E0 = np.ones(num_comparts)*9
U0 = np.ones(num_comparts)*10
I0 = np.ones(num_comparts)
CI0 = I0
H0 = np.zeros(num_comparts)
R0 = np.zeros(num_comparts)
UR0=np.zeros(num_comparts)
D0=np.zeros(num_comparts)
V0=np.zeros(num_comparts)
Vd0=np.zeros(num_comparts)
CR0=np.zeros(num_comparts)
S0 = N0 - I0 - R0 - H0 - U0 - E0 - UR0 - V0
saved = np.append(np.append(np.append(np.append(np.append(np.append(np.append(np.append(np.append(np.append(np.append(S0,E0),U0),I0),CI0),H0),R0),UR0),D0),V0),Vd0),CR0)

In [11]:
[s2,s_u2,gi1,gu1,gh1,eta_m,mi1,mu1,mh1,theta_eta,di1,du1,nu1,nuu1,nuuu1,eff1] = results['int1'].x
numbers['int1']=Model(s2,s_u2,gi1,gu1,gh1,eta_m,mi1,mu1,mh1,theta_eta,di1,du1,nu1,nuu1,nuuu1,eff1)
y=numbers['int1']
S = numbers['int1'][0:num_comparts*1]
E = y[num_comparts*1:num_comparts*2]
U = y[num_comparts*2:num_comparts*3]
I = y[num_comparts*3:num_comparts*4]
CI= y[num_comparts*4:num_comparts*5]
H = y[num_comparts*5:num_comparts*6]
R = y[num_comparts*6:num_comparts*7]
UR = y[num_comparts*7:num_comparts*8]
D = y[num_comparts*8:num_comparts*9]
V = y[num_comparts*9:num_comparts*10]
Vd = y[num_comparts*10:num_comparts*11]
CR = y[num_comparts*11:num_comparts*12]

In [12]:
pen=1
flowin = np.ones((num_comparts,num_comparts))
ffdata = pd.read_csv('data/region_flow_04_29_2020_05_17_2020_Weekday.csv')
ff=np.zeros((8,8))
ff[:,0]=ffdata['1.0'].values[0:8]
ff[:,1]=ffdata['2.0'].values[0:8]
ff[:,2]=ffdata['3.0'].values[0:8]
ff[:,3]=ffdata['4.0'].values[0:8]
ff[:,4]=ffdata['5.0'].values[0:8]
ff[:,5]=ffdata['6.0'].values[0:8]
ff[:,6]=ffdata['7.0'].values[0:8]
ff[:,7]=ffdata['8.0'].values[0:8]
list_of_health_dist={}
list_of_health_dist[0]=1
list_of_health_dist[1]=4+list_of_health_dist[0]
list_of_health_dist[2]=5+list_of_health_dist[1]
list_of_health_dist[3]=3+list_of_health_dist[2]
list_of_health_dist[4]=1+list_of_health_dist[3]
list_of_health_dist[5]=4+list_of_health_dist[4]
list_of_health_dist[6]=4+list_of_health_dist[5]
list_of_health_dist[7]=4+list_of_health_dist[6]

for j in range(num_comparts):
    for k in range(num_comparts):
        row = np.min(np.where([list_of_health_dist[ii]>j for ii in range(8)]))
        col = np.min(np.where([list_of_health_dist[ii]>k for ii in range(8)]))
        if row ==0:
            size_r = 1
        else:
            size_r = list_of_health_dist[row]-list_of_health_dist[row-1]
        if col ==0:
            size_c = 1
        else:
            size_c = list_of_health_dist[col]-list_of_health_dist[col-1]
        value = ff[row,col]/(size_c*size_r)
        flowin[j,k] = value


flowin = flowin/100
#flowin = np.ones((num_comparts,num_comparts))

In [13]:
saved = np.append(np.append(np.append(np.append(np.append(np.append(np.append(np.append(np.append(np.append(np.append(S.T[-1],E.T[-1]),U.T[-1]),I.T[-1]),CI.T[-1]),H.T[-1]),R.T[-1]),UR.T[-1]),D.T[-1]),V.T[-1]),Vd.T[-1]),CR.T[-1])

In [14]:
T2 = 45+46
int_=2
T1 = 45
dt = 1/24
d = np.genfromtxt ('data/death.csv', delimiter=",")
c = np.genfromtxt ('data/cases.csv', delimiter=",")
vv= np.genfromtxt ('data/vac.csv', delimiter=",")
h = np.genfromtxt ('data/hospitalization_data.csv',delimiter=",")
actual_case = np.zeros((T2,num_comparts))
case_daily = np.zeros((T2,num_comparts))
actual_death = np.zeros((T2,num_comparts))
actual_v = np.zeros((T2,num_comparts))
actual_hos = np.zeros((T2,num_comparts))
count = 0 
for i in range(0,T2):
    actual_case[i,:] = np.round(c[i,:],0)
    case_daily[i,:] = np.round(c[i,:],0)
    actual_death[i,:] = np.round(d[i,:],0)
    if i>=300:   
        actual_v[i,:] = np.round(vv[i-300,:],0)
    actual_hos[i] = np.round(h[i,:],0)
for i in range(0,T2):
    if i>0:
        actual_case[i,:] += actual_case[i-1,:]
        actual_death[i,:] += actual_death[i-1,:]
actual_case = actual_case[T1:T2]
case_daily = case_daily[T1:T2]
w_case_daily = case_daily.copy()
for i in range(case_daily.shape[1]):
    for t in range(6,case_daily.shape[0]):
        w_case_daily[t,i] = np.mean(case_daily[t-6:t+1,i])
actual_death = actual_death[T1:T2]
actual_hos = actual_hos[T1:T2]
actual_v = actual_v[T1:T2]
t = np.linspace(0, T2-T1-1, T2-T1)

In [15]:
[s2,s_u2,gi1,gu1,gh1,eta_m,mi1,mu1,mh1,theta_eta,di1,du1,nu1,nuu1,nuuu1,eff1] = results['int2'].x
numbers['int2']=Model(s2,s_u2,gi1,gu1,gh1,eta_m,mi1,mu1,mh1,theta_eta,di1,du1,nu1,nuu1,nuuu1,eff1)
y=numbers['int2']
S = numbers['int2'][0:num_comparts*1]
E = y[num_comparts*1:num_comparts*2]
U = y[num_comparts*2:num_comparts*3]
I = y[num_comparts*3:num_comparts*4]
CI= y[num_comparts*4:num_comparts*5]
H = y[num_comparts*5:num_comparts*6]
R = y[num_comparts*6:num_comparts*7]
UR = y[num_comparts*7:num_comparts*8]
D = y[num_comparts*8:num_comparts*9]
V = y[num_comparts*9:num_comparts*10]
Vd = y[num_comparts*10:num_comparts*11]
CR = y[num_comparts*11:num_comparts*12]

In [16]:
flowin = np.ones((num_comparts,num_comparts))
ffdata = pd.read_csv('data/region_flow_06_05_2020_06_25_2020_Weekday.csv')
ff=np.zeros((8,8))
ff[:,0]=ffdata['1.0'].values[0:8]
ff[:,1]=ffdata['2.0'].values[0:8]
ff[:,2]=ffdata['3.0'].values[0:8]
ff[:,3]=ffdata['4.0'].values[0:8]
ff[:,4]=ffdata['5.0'].values[0:8]
ff[:,5]=ffdata['6.0'].values[0:8]
ff[:,6]=ffdata['7.0'].values[0:8]
ff[:,7]=ffdata['8.0'].values[0:8]
list_of_health_dist={}
list_of_health_dist[0]=1
list_of_health_dist[1]=4+list_of_health_dist[0]
list_of_health_dist[2]=5+list_of_health_dist[1]
list_of_health_dist[3]=3+list_of_health_dist[2]
list_of_health_dist[4]=1+list_of_health_dist[3]
list_of_health_dist[5]=4+list_of_health_dist[4]
list_of_health_dist[6]=4+list_of_health_dist[5]
list_of_health_dist[7]=4+list_of_health_dist[6]

for j in range(num_comparts):
    for k in range(num_comparts):
        row = np.min(np.where([list_of_health_dist[ii]>j for ii in range(8)]))
        col = np.min(np.where([list_of_health_dist[ii]>k for ii in range(8)]))
        if row ==0:
            size_r = 1
        else:                                                                                                                                                                                                                                                                                                                  
            size_r = list_of_health_dist[row]-list_of_health_dist[row-1]
        if col ==0:
            size_c = 1
        else:
            size_c = list_of_health_dist[col]-list_of_health_dist[col-1]
        value = ff[row,col]/(size_c*size_r)
        flowin[j,k] = value


flowin = flowin/100
#flowin = np.ones((num_comparts,num_comparts))
saved = np.append(np.append(np.append(np.append(np.append(np.append(np.append(np.append(np.append(np.append(np.append(S.T[-1],E.T[-1]),U.T[-1]),I.T[-1]),CI.T[-1]),H.T[-1]),R.T[-1]),UR.T[-1]),D.T[-1]),V.T[-1]),Vd.T[-1]),CR.T[-1])

In [17]:
T2 = 45+45+46
T1 = 45+45
int_=3
dt = 1/24
d = np.genfromtxt ('data/death.csv', delimiter=",")
c = np.genfromtxt ('data/cases.csv', delimiter=",")
vv= np.genfromtxt ('data/vac.csv', delimiter=",")
h = np.genfromtxt ('data/hospitalization_data.csv',delimiter=",")
actual_case = np.zeros((T2,num_comparts))
case_daily = np.zeros((T2,num_comparts))
actual_death = np.zeros((T2,num_comparts))
actual_v = np.zeros((T2,num_comparts))
actual_hos = np.zeros((T2,num_comparts))
count = 0 
for i in range(0,T2):
    actual_case[i,:] = np.round(c[i,:],0)
    case_daily[i,:] = np.round(c[i,:],0)
    actual_death[i,:] = np.round(d[i,:],0)
    if i>=300:   
        actual_v[i,:] = np.round(vv[i-300,:],0)
    actual_hos[i] = np.round(h[i,:],0)
for i in range(0,T2):
    if i>0:
        actual_case[i,:] += actual_case[i-1,:]
        actual_death[i,:] += actual_death[i-1,:]
actual_case = actual_case[T1:T2]
case_daily = case_daily[T1:T2]
w_case_daily = case_daily.copy()
for i in range(case_daily.shape[1]):
    for t in range(6,case_daily.shape[0]):
        w_case_daily[t,i] = np.mean(case_daily[t-6:t+1,i])
actual_death = actual_death[T1:T2]
actual_hos = actual_hos[T1:T2]
actual_v = actual_v[T1:T2]
t = np.linspace(0, T2-T1-1, T2-T1)

In [18]:
[s2,s_u2,gi1,gu1,gh1,eta_m,mi1,mu1,mh1,theta_eta,di1,du1,nu1,nuu1,nuuu1,eff1] = results['int3'].x
numbers['int3']=Model(s2,s_u2,gi1,gu1,gh1,eta_m,mi1,mu1,mh1,theta_eta,di1,du1,nu1,nuu1,nuuu1,eff1)
y=numbers['int3']
S = numbers['int3'][0:num_comparts*1]
E = y[num_comparts*1:num_comparts*2]
U = y[num_comparts*2:num_comparts*3]
I = y[num_comparts*3:num_comparts*4]
CI= y[num_comparts*4:num_comparts*5]
H = y[num_comparts*5:num_comparts*6]
R = y[num_comparts*6:num_comparts*7]
UR = y[num_comparts*7:num_comparts*8]
D = y[num_comparts*8:num_comparts*9]
V = y[num_comparts*9:num_comparts*10]
Vd = y[num_comparts*10:num_comparts*11]
CR = y[num_comparts*11:num_comparts*12]

In [19]:
flowin = np.ones((num_comparts,num_comparts))
ffdata = pd.read_csv('data/region_flow_09_16_2020_09_30_2020_Weekday.csv')
ff=np.zeros((8,8))
ff[:,0]=ffdata['1.0'].values[0:8]
ff[:,1]=ffdata['2.0'].values[0:8]
ff[:,2]=ffdata['3.0'].values[0:8]
ff[:,3]=ffdata['4.0'].values[0:8]
ff[:,4]=ffdata['5.0'].values[0:8]
ff[:,5]=ffdata['6.0'].values[0:8]
ff[:,6]=ffdata['7.0'].values[0:8]
ff[:,7]=ffdata['8.0'].values[0:8]
list_of_health_dist={}
list_of_health_dist[0]=1
list_of_health_dist[1]=4+list_of_health_dist[0]
list_of_health_dist[2]=5+list_of_health_dist[1]
list_of_health_dist[3]=3+list_of_health_dist[2]
list_of_health_dist[4]=1+list_of_health_dist[3]
list_of_health_dist[5]=4+list_of_health_dist[4]
list_of_health_dist[6]=4+list_of_health_dist[5]
list_of_health_dist[7]=4+list_of_health_dist[6]

for j in range(num_comparts):
    for k in range(num_comparts):
        row = np.min(np.where([list_of_health_dist[ii]>j for ii in range(8)]))
        col = np.min(np.where([list_of_health_dist[ii]>k for ii in range(8)]))
        if row ==0:
            size_r = 1
        else:
            size_r = list_of_health_dist[row]-list_of_health_dist[row-1]
        if col ==0:
            size_c = 1
        else:
            size_c = list_of_health_dist[col]-list_of_health_dist[col-1]
        value = ff[row,col]/(size_c*size_r)
        flowin[j,k] = value


flowin = flowin/100
#flowin = np.ones((num_comparts,num_comparts))
saved = np.append(np.append(np.append(np.append(np.append(np.append(np.append(np.append(np.append(np.append(np.append(S.T[-1],E.T[-1]),U.T[-1]),I.T[-1]),CI.T[-1]),H.T[-1]),R.T[-1]),UR.T[-1]),D.T[-1]),V.T[-1]),Vd.T[-1]),CR.T[-1])

In [20]:
T2 = 45+45+45+76
T1 = 45+45+45
int_=4
dt = 1/24
d = np.genfromtxt ('data/death.csv', delimiter=",")
c = np.genfromtxt ('data/cases.csv', delimiter=",")
vv= np.genfromtxt ('data/vac.csv', delimiter=",")
h = np.genfromtxt ('data/hospitalization_data.csv',delimiter=",")
actual_case = np.zeros((T2,num_comparts))
case_daily = np.zeros((T2,num_comparts))
actual_death = np.zeros((T2,num_comparts))
actual_v = np.zeros((T2,num_comparts))
actual_hos = np.zeros((T2,num_comparts))
count = 0 
for i in range(0,T2):
    actual_case[i,:] = np.round(c[i,:],0)
    case_daily[i,:] = np.round(c[i,:],0)
    actual_death[i,:] = np.round(d[i,:],0)
    if i>=300:   
        actual_v[i,:] = np.round(vv[i-300,:],0)
    actual_hos[i] = np.round(h[i,:],0)
for i in range(0,T2):
    if i>0:
        actual_case[i,:] += actual_case[i-1,:]
        actual_death[i,:] += actual_death[i-1,:]
actual_case = actual_case[T1:T2]
case_daily = case_daily[T1:T2]
w_case_daily = case_daily.copy()
for i in range(case_daily.shape[1]):
    for t in range(6,case_daily.shape[0]):
        w_case_daily[t,i] = np.mean(case_daily[t-6:t+1,i])
actual_death = actual_death[T1:T2]
actual_hos = actual_hos[T1:T2]
actual_v = actual_v[T1:T2]
t = np.linspace(0, T2-T1-1, T2-T1)

In [21]:
[s2,s_u2,gi1,gu1,gh1,eta_m,mi1,mu1,mh1,theta_eta,di1,du1,nu1,nuu1,nuuu1,eff1] = results['int4'].x
numbers['int4']=Model(s2,s_u2,gi1,gu1,gh1,eta_m,mi1,mu1,mh1,theta_eta,di1,du1,nu1,nuu1,nuuu1,eff1)
y=numbers['int4']
S = numbers['int4'][0:num_comparts*1]
E = y[num_comparts*1:num_comparts*2]
U = y[num_comparts*2:num_comparts*3]
I = y[num_comparts*3:num_comparts*4]
CI= y[num_comparts*4:num_comparts*5]
H = y[num_comparts*5:num_comparts*6]
R = y[num_comparts*6:num_comparts*7]
UR = y[num_comparts*7:num_comparts*8]
D = y[num_comparts*8:num_comparts*9]
V = y[num_comparts*9:num_comparts*10]
Vd = y[num_comparts*10:num_comparts*11]
CR = y[num_comparts*11:num_comparts*12]

In [22]:
flowin = np.ones((num_comparts,num_comparts))
ffdata = pd.read_csv('data/region_flow_11_16_2020_11_30_2020_Weekday.csv')
ff=np.zeros((8,8))
ff[:,0]=ffdata['1.0'].values[0:8]
ff[:,1]=ffdata['2.0'].values[0:8]
ff[:,2]=ffdata['3.0'].values[0:8]
ff[:,3]=ffdata['4.0'].values[0:8]
ff[:,4]=ffdata['5.0'].values[0:8]
ff[:,5]=ffdata['6.0'].values[0:8]
ff[:,6]=ffdata['7.0'].values[0:8]
ff[:,7]=ffdata['8.0'].values[0:8]
list_of_health_dist={}
list_of_health_dist[0]=1
list_of_health_dist[1]=4+list_of_health_dist[0]
list_of_health_dist[2]=5+list_of_health_dist[1]
list_of_health_dist[3]=3+list_of_health_dist[2]
list_of_health_dist[4]=1+list_of_health_dist[3]
list_of_health_dist[5]=4+list_of_health_dist[4]
list_of_health_dist[6]=4+list_of_health_dist[5]
list_of_health_dist[7]=4+list_of_health_dist[6]

for j in range(num_comparts):
    for k in range(num_comparts):
        row = np.min(np.where([list_of_health_dist[ii]>j for ii in range(8)]))
        col = np.min(np.where([list_of_health_dist[ii]>k for ii in range(8)]))
        if row ==0:
            size_r = 1
        else:
            size_r = list_of_health_dist[row]-list_of_health_dist[row-1]
        if col ==0:
            size_c = 1
        else:
            size_c = list_of_health_dist[col]-list_of_health_dist[col-1]
        value = ff[row,col]/(size_c*size_r)
        flowin[j,k] = value


flowin = flowin/100
#flowin = np.ones((num_comparts,num_comparts))
saved = np.append(np.append(np.append(np.append(np.append(np.append(np.append(np.append(np.append(np.append(np.append(S.T[-1],E.T[-1]),U.T[-1]),I.T[-1]),CI.T[-1]),H.T[-1]),R.T[-1]),UR.T[-1]),D.T[-1]),V.T[-1]),Vd.T[-1]),CR.T[-1])

In [23]:

T2 = 45+45+45+75+91
T1 = 45+45+45+75
int_ = 5
dt = 1/24
d = np.genfromtxt ('data/death.csv', delimiter=",")
c = np.genfromtxt ('data/cases.csv', delimiter=",")
vv= np.genfromtxt ('data/vac.csv', delimiter=",")
h = np.genfromtxt ('data/hospitalization_data.csv',delimiter=",")
actual_case = np.zeros((T2,num_comparts))
case_daily = np.zeros((T2,num_comparts))
actual_death = np.zeros((T2,num_comparts))
actual_v = np.zeros((T2,num_comparts))
actual_hos = np.zeros((T2,num_comparts))
count = 0 
for i in range(0,T2):
    actual_case[i,:] = np.round(c[i,:],0)
    case_daily[i,:] = np.round(c[i,:],0)
    actual_death[i,:] = np.round(d[i,:],0)
    if i>=300:   
        actual_v[i,:] = np.round(vv[i-300,:],0)
    actual_hos[i] = np.round(h[i,:],0)
for i in range(0,T2):
    if i>0:
        actual_case[i,:] += actual_case[i-1,:]
        actual_death[i,:] += actual_death[i-1,:]
actual_case = actual_case[T1:T2]
case_daily = case_daily[T1:T2]
w_case_daily = case_daily.copy()
for i in range(case_daily.shape[1]):
    for t in range(6,case_daily.shape[0]):
        w_case_daily[t,i] = np.mean(case_daily[t-6:t+1,i])
actual_death = actual_death[T1:T2]
actual_hos = actual_hos[T1:T2]
actual_v = actual_v[T1:T2]
t = np.linspace(0, T2-T1-1, T2-T1)

In [24]:
[s2,s_u2,gi1,gu1,gh1,eta_m,mi1,mu1,mh1,theta_eta,di1,du1,nu1,nuu1,nuuu1,eff1] = results['int5'].x
numbers['int5']=Model(s2,s_u2,gi1,gu1,gh1,eta_m,mi1,mu1,mh1,theta_eta,di1,du1,nu1,nuu1,nuuu1,eff1)
y=numbers['int5']
S = numbers['int5'][0:num_comparts*1]
E = y[num_comparts*1:num_comparts*2]
U = y[num_comparts*2:num_comparts*3]
I = y[num_comparts*3:num_comparts*4]
CI= y[num_comparts*4:num_comparts*5]
H = y[num_comparts*5:num_comparts*6]
R = y[num_comparts*6:num_comparts*7]
UR = y[num_comparts*7:num_comparts*8]
D = y[num_comparts*8:num_comparts*9]
V = y[num_comparts*9:num_comparts*10]
Vd = y[num_comparts*10:num_comparts*11]
CR = y[num_comparts*11:num_comparts*12]

In [25]:
S1=numbers['int1'][num_comparts*0:num_comparts*1]
S2=numbers['int2'][num_comparts*0:num_comparts*1]
S3=numbers['int3'][num_comparts*0:num_comparts*1]
S4=numbers['int4'][num_comparts*0:num_comparts*1]
S5=numbers['int5'][num_comparts*0:num_comparts*1]
#S6=numbers['int6'][num_comparts*0:num_comparts*1]
#S7=numbers['int7'][num_comparts*0:num_comparts*1]
#S8=numbers['int8'][num_comparts*0:num_comparts*1]
#S9=numbers['int9'][num_comparts*0:num_comparts*1]
#S10=numbers['int10'][num_comparts*0:num_comparts*1]
#S11=numbers['int11'][num_comparts*0:num_comparts*1]
#S12=numbers['int12'][num_comparts*0:num_comparts*1]
#S13=numbers['int13'][num_comparts*0:num_comparts*1]
S1 =np.sum(S1,axis=0)
S2 =np.sum(S2,axis=0)
S3 =np.sum(S3,axis=0)
S4 =np.sum(S4,axis=0)
S5 =np.sum(S5,axis=0)
#S6 =np.sum(S6,axis=0)
#S7 =np.sum(S7,axis=0)
#S8 =np.sum(S8,axis=0)
#S9 =np.sum(S9,axis=0)
#S10 =np.sum(S10,axis=0)
#S11 =np.sum(S11,axis=0)
#S12 =np.sum(S12,axis=0)
#S13 =np.sum(S13,axis=0)
S = np.append(S1[0:45],S2)
S = np.append(S[0:-1],S3)
S = np.append(S[0:-1],S4)
S = np.append(S[0:-1],S5)
#S = np.append(S[0:-1],S6)
#S = np.append(S[0:-1],S7)
#S = np.append(S[0:-1],S8)
#S = np.append(S[0:-1],S9)
#S = np.append(S[0:-1],S10)
#S = np.append(S[0:-1],S11)
#S = np.append(S[0:-1],S12)
#S = np.append(S[0:-1],S13)

R1=numbers['int1'][num_comparts*11:num_comparts*12]
R2=numbers['int2'][num_comparts*11:num_comparts*12]
R3=numbers['int3'][num_comparts*11:num_comparts*12]
R4=numbers['int4'][num_comparts*11:num_comparts*12]
R5=numbers['int5'][num_comparts*11:num_comparts*12]
#R6=numbers['int6'][num_comparts*11:num_comparts*12]
#R7=numbers['int7'][num_comparts*11:num_comparts*12]
#R8=numbers['int8'][num_comparts*11:num_comparts*12]
#R9=numbers['int9'][num_comparts*11:num_comparts*12]
#R10=numbers['int10'][num_comparts*11:num_comparts*12]
#R11=numbers['int11'][num_comparts*11:num_comparts*12]
#R12=numbers['int12'][num_comparts*11:num_comparts*12]
#R13=numbers['int13'][num_comparts*11:num_comparts*12]
R1 =np.sum(R1,axis=0)
R2 =np.sum(R2,axis=0)
R3 =np.sum(R3,axis=0)
R4 =np.sum(R4,axis=0)
R5 =np.sum(R5,axis=0)
#R6 =np.sum(R6,axis=0)
#R7 =np.sum(R7,axis=0)
#R8 =np.sum(R8,axis=0)
#R9 =np.sum(R9,axis=0)
#R10 =np.sum(R10,axis=0)
#R11 =np.sum(R11,axis=0)
#R12 =np.sum(R12,axis=0)
#R13 =np.sum(R13,axis=0)
R = np.append(R1[0:45],R2)
R = np.append(R[0:-1],R3)
R = np.append(R[0:-1],R4)
R = np.append(R[0:-1],R5)
#R = np.append(R[0:-1],R6)
#R = np.append(R[0:-1],R7)
#R = np.append(R[0:-1],R8)
#R = np.append(R[0:-1],R9)
#R = np.append(R[0:-1],R10)
#R = np.append(R[0:-1],R11)
#R = np.append(R[0:-1],R12)
#R = np.append(R[0:-1],R13)


H1=numbers['int1'][num_comparts*5:num_comparts*6]
H2=numbers['int2'][num_comparts*5:num_comparts*6]
H3=numbers['int3'][num_comparts*5:num_comparts*6]
H4=numbers['int4'][num_comparts*5:num_comparts*6]
H5=numbers['int5'][num_comparts*5:num_comparts*6]
#H6=numbers['int6'][num_comparts*5:num_comparts*6]
#H7=numbers['int7'][num_comparts*5:num_comparts*6]
#H8=numbers['int8'][num_comparts*5:num_comparts*6]
#H9=numbers['int9'][num_comparts*5:num_comparts*6]
#H10=numbers['int10'][num_comparts*5:num_comparts*6]
#H11=numbers['int11'][num_comparts*5:num_comparts*6]
#H12=numbers['int12'][num_comparts*5:num_comparts*6]
#H13=numbers['int13'][num_comparts*5:num_comparts*6]
H1 =np.sum(H1,axis=0)
H2 =np.sum(H2,axis=0)
H3 =np.sum(H3,axis=0)
H4 =np.sum(H4,axis=0)
H5 =np.sum(H5,axis=0)
#H6 =np.sum(H6,axis=0)
#H7 =np.sum(H7,axis=0)
#H8 =np.sum(H8,axis=0)
#H9 =np.sum(H9,axis=0)
#H10 =np.sum(H10,axis=0)
#H11 =np.sum(H11,axis=0)
#H12 =np.sum(H12,axis=0)
#H13 =np.sum(H13,axis=0)
H = np.append(H1[0:45],H2)
H = np.append(H[0:-1],H3)
H = np.append(H[0:-1],H4)
H = np.append(H[0:-1],H5)
#H = np.append(H[0:-1],H6)
#H = np.append(H[0:-1],H7)
#H = np.append(H[0:-1],H8)
#H = np.append(H[0:-1],H9)
#H = np.append(H[0:-1],H10)
#H = np.append(H[0:-1],H11)
#H = np.append(H[0:-1],H12)
#H = np.append(H[0:-1],H13)

UR1=numbers['int1'][num_comparts*7:num_comparts*8]
UR2=numbers['int2'][num_comparts*7:num_comparts*8]
UR3=numbers['int3'][num_comparts*7:num_comparts*8]
UR4=numbers['int4'][num_comparts*7:num_comparts*8]
UR5=numbers['int5'][num_comparts*7:num_comparts*8]
#UR6=numbers['int6'][num_comparts*7:num_comparts*8]
#UR7=numbers['int7'][num_comparts*7:num_comparts*8]
#UR8=numbers['int8'][num_comparts*7:num_comparts*8]
#UR9=numbers['int9'][num_comparts*7:num_comparts*8]
#UR10=numbers['int10'][num_comparts*7:num_comparts*8]
#UR11=numbers['int11'][num_comparts*7:num_comparts*8]
#UR12=numbers['int12'][num_comparts*7:num_comparts*8]
#UR13=numbers['int13'][num_comparts*7:num_comparts*8]
UR1 =np.sum(UR1,axis=0)
UR2 =np.sum(UR2,axis=0)
UR3 =np.sum(UR3,axis=0)
UR4 =np.sum(UR4,axis=0)
UR5 =np.sum(UR5,axis=0)
#UR6 =np.sum(UR6,axis=0)
#UR7 =np.sum(UR7,axis=0)
#UR8 =np.sum(UR8,axis=0)
#UR9 =np.sum(UR9,axis=0)
#UR10 =np.sum(UR10,axis=0)
#UR11 =np.sum(UR11,axis=0)
#UR12 =np.sum(UR12,axis=0)
#UR13 =np.sum(UR13,axis=0)
UR = np.append(UR1[0:45],UR2)
UR = np.append(UR[0:-1],UR3)
UR = np.append(UR[0:-1],UR4)
UR = np.append(UR[0:-1],UR5)
#UR = np.append(UR[0:-1],UR6)
#UR = np.append(UR[0:-1],UR7)
#UR = np.append(UR[0:-1],UR8)
#UR = np.append(UR[0:-1],UR9)
#UR = np.append(UR[0:-1],UR10)
#UR = np.append(UR[0:-1],UR11)
#UR = np.append(UR[0:-1],UR12)
#UR = np.append(UR[0:-1],UR13)

D1=numbers['int1'][num_comparts*8:num_comparts*9]
D2=numbers['int2'][num_comparts*8:num_comparts*9]
D3=numbers['int3'][num_comparts*8:num_comparts*9]
D4=numbers['int4'][num_comparts*8:num_comparts*9]
D5=numbers['int5'][num_comparts*8:num_comparts*9]
#D6=numbers['int6'][num_comparts*8:num_comparts*9]
#D7=numbers['int7'][num_comparts*8:num_comparts*9]
#D8=numbers['int8'][num_comparts*8:num_comparts*9]
#D9=numbers['int9'][num_comparts*8:num_comparts*9]
#D10=numbers['int10'][num_comparts*8:num_comparts*9]
#D11=numbers['int11'][num_comparts*8:num_comparts*9]
#D12=numbers['int12'][num_comparts*8:num_comparts*9]
#D13=numbers['int13'][num_comparts*8:num_comparts*9]
D1 =np.sum(D1,axis=0)
D2 =np.sum(D2,axis=0)
D3 =np.sum(D3,axis=0)
D4 =np.sum(D4,axis=0)
D5 =np.sum(D5,axis=0)
#D6 =np.sum(D6,axis=0)
#D7 =np.sum(D7,axis=0)
#D8 =np.sum(D8,axis=0)
#D9 =np.sum(D9,axis=0)
#D10 =np.sum(D10,axis=0)
#D11 =np.sum(D11,axis=0)
#D12 =np.sum(D12,axis=0)
#D13 =np.sum(D13,axis=0)
D = np.append(D1[0:45],D2)
D = np.append(D[0:-1],D3)
D = np.append(D[0:-1],D4)
D = np.append(D[0:-1],D5)
#D = np.append(D[0:-1],D6)
#D = np.append(D[0:-1],D7)
#D = np.append(D[0:-1],D8)
#D = np.append(D[0:-1],D9)
#D = np.append(D[0:-1],D10)
#D = np.append(D[0:-1],D11)
#D = np.append(D[0:-1],D12)
#D = np.append(D[0:-1],D13)

C1=numbers['int1'][num_comparts*4:num_comparts*5]
C2=numbers['int2'][num_comparts*4:num_comparts*5]
C3=numbers['int3'][num_comparts*4:num_comparts*5]
C4=numbers['int4'][num_comparts*4:num_comparts*5]
C5=numbers['int5'][num_comparts*4:num_comparts*5]
#C6=numbers['int6'][num_comparts*4:num_comparts*5]
#C7=numbers['int7'][num_comparts*4:num_comparts*5]
#C8=numbers['int8'][num_comparts*4:num_comparts*5]
#C9=numbers['int9'][num_comparts*4:num_comparts*5]
#C10=numbers['int10'][num_comparts*4:num_comparts*5]
#C11=numbers['int11'][num_comparts*4:num_comparts*5]
#C12=numbers['int12'][num_comparts*4:num_comparts*5]
#C13=numbers['int13'][num_comparts*4:num_comparts*5]
C1 =np.sum(C1,axis=0)
C2 =np.sum(C2,axis=0)
C3 =np.sum(C3,axis=0)
C4 =np.sum(C4,axis=0)
C5 =np.sum(C5,axis=0)
#C6 =np.sum(C6,axis=0)
#C7 =np.sum(C7,axis=0)
#C8 =np.sum(C8,axis=0)
#C9 =np.sum(C9,axis=0)
#C10 =np.sum(C10,axis=0)
#C11 =np.sum(C11,axis=0)
#C12 =np.sum(C12,axis=0)
#C13 =np.sum(C13,axis=0)
C = np.append(C1[0:45],C2)
C = np.append(C[0:-1],C3)
C = np.append(C[0:-1],C4)
C = np.append(C[0:-1],C5)
#C = np.append(C[0:-1],C6)
#C = np.append(C[0:-1],C7)
#C = np.append(C[0:-1],C8)
#C = np.append(C[0:-1],C9)
#C = np.append(C[0:-1],C10)
#C = np.append(C[0:-1],C11)
#C = np.append(C[0:-1],C12)
#C = np.append(C[0:-1],C13)

U1=numbers['int1'][num_comparts*2:num_comparts*3]
U2=numbers['int2'][num_comparts*2:num_comparts*3]
U3=numbers['int3'][num_comparts*2:num_comparts*3]
U4=numbers['int4'][num_comparts*2:num_comparts*3]
U5=numbers['int5'][num_comparts*2:num_comparts*3]
#U6=numbers['int6'][num_comparts*2:num_comparts*3]
#U7=numbers['int7'][num_comparts*2:num_comparts*3]
#U8=numbers['int8'][num_comparts*2:num_comparts*3]
#U9=numbers['int9'][num_comparts*2:num_comparts*3]
#U10=numbers['int10'][num_comparts*2:num_comparts*3]
#U11=numbers['int11'][num_comparts*2:num_comparts*3]
#U12=numbers['int12'][num_comparts*2:num_comparts*3]
#U13=numbers['int13'][num_comparts*2:num_comparts*3]
U1 =np.sum(U1,axis=0)
U2 =np.sum(U2,axis=0)
U3 =np.sum(U3,axis=0)
U4 =np.sum(U4,axis=0)
U5 =np.sum(U5,axis=0)
#U6 =np.sum(U6,axis=0)
#U7 =np.sum(U7,axis=0)
#U8 =np.sum(U8,axis=0)
#U9 =np.sum(U9,axis=0)
#U10 =np.sum(U10,axis=0)
#U11 =np.sum(U11,axis=0)
#U12 =np.sum(U12,axis=0)
#U13 =np.sum(U13,axis=0)
U = np.append(U1[0:45],U2)
U = np.append(U[0:-1],U3)
U = np.append(U[0:-1],U4)
U = np.append(U[0:-1],U5)
#U = np.append(U[0:-1],U6)
#U = np.append(U[0:-1],U7)
#U = np.append(U[0:-1],U8)
#U = np.append(U[0:-1],U9)
#U = np.append(U[0:-1],U10)
#U = np.append(U[0:-1],U11)
#U = np.append(U[0:-1],U12)
#U = np.append(U[0:-1],U13)

E1=numbers['int1'][num_comparts*1:num_comparts*2]
E2=numbers['int2'][num_comparts*1:num_comparts*2]
E3=numbers['int3'][num_comparts*1:num_comparts*2]
E4=numbers['int4'][num_comparts*1:num_comparts*2]
E5=numbers['int5'][num_comparts*1:num_comparts*2]
#E6=numbers['int6'][num_comparts*1:num_comparts*2]
#E7=numbers['int7'][num_comparts*1:num_comparts*2]
#E8=numbers['int8'][num_comparts*1:num_comparts*2]
#E9=numbers['int9'][num_comparts*1:num_comparts*2]
#E10=numbers['int10'][num_comparts*1:num_comparts*2]
#E11=numbers['int11'][num_comparts*1:num_comparts*2]
#E12=numbers['int12'][num_comparts*1:num_comparts*2]
#E13=numbers['int13'][num_comparts*1:num_comparts*2]
E1 =np.sum(E1,axis=0)
E2 =np.sum(E2,axis=0)
E3 =np.sum(E3,axis=0)
E4 =np.sum(E4,axis=0)
E5 =np.sum(E5,axis=0)
#E6 =np.sum(E6,axis=0)
#E7 =np.sum(E7,axis=0)
#E8 =np.sum(E8,axis=0)
#E9 =np.sum(E9,axis=0)
#E10 =np.sum(E10,axis=0)
#E11 =np.sum(E11,axis=0)
#E12 =np.sum(E12,axis=0)
#E13 =np.sum(E13,axis=0)
E = np.append(E1[0:45],E2)
E = np.append(E[0:-1],E3)
E = np.append(E[0:-1],E4)
E = np.append(E[0:-1],E5)
#E = np.append(E[0:-1],E6)
#E = np.append(E[0:-1],E7)
#E = np.append(E[0:-1],E8)
#E = np.append(E[0:-1],E9)
#E = np.append(E[0:-1],E10)
#E = np.append(E[0:-1],E11)
#E = np.append(E[0:-1],E12)
#E = np.append(E[0:-1],E13)

I1=numbers['int1'][num_comparts*3:num_comparts*4]
I2=numbers['int2'][num_comparts*3:num_comparts*4]
I3=numbers['int3'][num_comparts*3:num_comparts*4]
I4=numbers['int4'][num_comparts*3:num_comparts*4]
I5=numbers['int5'][num_comparts*3:num_comparts*4]
#I6=numbers['int6'][num_comparts*3:num_comparts*4]
#I7=numbers['int7'][num_comparts*3:num_comparts*4]
#I8=numbers['int8'][num_comparts*3:num_comparts*4]
#I9=numbers['int9'][num_comparts*3:num_comparts*4]
#I10=numbers['int10'][num_comparts*3:num_comparts*4]
#I11=numbers['int11'][num_comparts*3:num_comparts*4]
#I12=numbers['int12'][num_comparts*3:num_comparts*4]
#I13=numbers['int13'][num_comparts*3:num_comparts*4]
I1 =np.sum(I1,axis=0)
I2 =np.sum(I2,axis=0)
I3 =np.sum(I3,axis=0)
I4 =np.sum(I4,axis=0)
I5 =np.sum(I5,axis=0)
#I6 =np.sum(I6,axis=0)
#I7 =np.sum(I7,axis=0)
#I8 =np.sum(I8,axis=0)
#I9 =np.sum(I9,axis=0)
#I10 =np.sum(I10,axis=0)
#I11 =np.sum(I11,axis=0)
#I12 =np.sum(I12,axis=0)
#I13 =np.sum(I13,axis=0)
I = np.append(I1[0:45],I2)
I = np.append(I[0:-1],I3)
I = np.append(I[0:-1],I4)
I = np.append(I[0:-1],I5)
#I = np.append(I[0:-1],I6)
#I = np.append(I[0:-1],I7)
#I = np.append(I[0:-1],I8)
#I = np.append(I[0:-1],I9)
#I = np.append(I[0:-1],I10)
#I = np.append(I[0:-1],I11)
#I = np.append(I[0:-1],I12)
#I = np.append(I[0:-1],I13)

V1=numbers['int1'][num_comparts*10:num_comparts*11]
V2=numbers['int2'][num_comparts*10:num_comparts*11]
V3=numbers['int3'][num_comparts*10:num_comparts*11]
V4=numbers['int4'][num_comparts*10:num_comparts*11]
V5=numbers['int5'][num_comparts*10:num_comparts*11]
#V6=numbers['int6'][num_comparts*10:num_comparts*11]
#V7=numbers['int7'][num_comparts*10:num_comparts*11]
#V8=numbers['int8'][num_comparts*10:num_comparts*11]
#V9=numbers['int9'][num_comparts*10:num_comparts*11]
#V10=numbers['int10'][num_comparts*10:num_comparts*11]
#V11=numbers['int11'][num_comparts*10:num_comparts*11]
#V12=numbers['int12'][num_comparts*10:num_comparts*11]
#V13=numbers['int13'][num_comparts*10:num_comparts*11]
V1 =np.sum(V1,axis=0)
V2 =np.sum(V2,axis=0)
V3 =np.sum(V3,axis=0)
V4 =np.sum(V4,axis=0)
V5 =np.sum(V5,axis=0)
#V6 =np.sum(V6,axis=0)
#V7 =np.sum(V7,axis=0)
#V8 =np.sum(V8,axis=0)
#V9 =np.sum(V9,axis=0)
#V10 =np.sum(V10,axis=0)
#V11 =np.sum(V11,axis=0)
#V12 =np.sum(V12,axis=0)
#V13 =np.sum(V13,axis=0)
V = np.append(V1[0:45],V2)
V = np.append(V[0:-1],V3)
V = np.append(V[0:-1],V4)
V = np.append(V[0:-1],V5)
#V = np.append(V[0:-1],V6)
#V = np.append(V[0:-1],V7)
#V = np.append(V[0:-1],V8)
#V = np.append(V[0:-1],V9)
#V = np.append(V[0:-1],V10)
#V = np.append(V[0:-1],V11)
#V = np.append(V[0:-1],V12)
#V = np.append(V[0:-1],V13)

Vt1=numbers['int1'][num_comparts*9:num_comparts*10]
Vt2=numbers['int2'][num_comparts*9:num_comparts*10]
Vt3=numbers['int3'][num_comparts*9:num_comparts*10]
Vt4=numbers['int4'][num_comparts*9:num_comparts*10]
Vt5=numbers['int5'][num_comparts*9:num_comparts*10]
#Vt6=numbers['int6'][num_comparts*9:num_comparts*10]
#Vt7=numbers['int7'][num_comparts*9:num_comparts*10]
#Vt8=numbers['int8'][num_comparts*9:num_comparts*10]
#Vt9=numbers['int9'][num_comparts*9:num_comparts*10]
#Vt10=numbers['int10'][num_comparts*9:num_comparts*10]
#Vt11=numbers['int11'][num_comparts*9:num_comparts*10]
#Vt12=numbers['int12'][num_comparts*9:num_comparts*10]
#Vt13=numbers['int13'][num_comparts*9:num_comparts*10]
Vt1 =np.sum(Vt1,axis=0)
Vt2 =np.sum(Vt2,axis=0)
Vt3 =np.sum(Vt3,axis=0)
Vt4 =np.sum(Vt4,axis=0)
Vt5 =np.sum(Vt5,axis=0)
#Vt6 =np.sum(Vt6,axis=0)
#Vt7 =np.sum(Vt7,axis=0)
#Vt8 =np.sum(Vt8,axis=0)
#Vt9 =np.sum(Vt9,axis=0)
#Vt10 =np.sum(Vt10,axis=0)
#Vt11 =np.sum(Vt11,axis=0)
#Vt12 =np.sum(Vt12,axis=0)
#Vt13 =np.sum(Vt13,axis=0)
Vt = np.append(Vt1[0:45],Vt2)
Vt = np.append(Vt[0:-1],Vt3)
Vt = np.append(Vt[0:-1],Vt4)
Vt = np.append(Vt[0:-1],Vt5)
#Vt = np.append(Vt[0:-1],Vt6)
#Vt = np.append(Vt[0:-1],Vt7)
#Vt = np.append(Vt[0:-1],Vt8)
#Vt = np.append(Vt[0:-1],Vt9)
#Vt = np.append(Vt[0:-1],Vt10)
#Vt = np.append(Vt[0:-1],Vt11)
#Vt = np.append(Vt[0:-1],Vt12)
#Vt = np.append(Vt[0:-1],Vt13)

Rt1=numbers['int1'][num_comparts*6:num_comparts*7]
Rt2=numbers['int2'][num_comparts*6:num_comparts*7]
Rt3=numbers['int3'][num_comparts*6:num_comparts*7]
Rt4=numbers['int4'][num_comparts*6:num_comparts*7]
Rt5=numbers['int5'][num_comparts*6:num_comparts*7]
#Rt6=numbers['int6'][num_comparts*6:num_comparts*7]
#Rt7=numbers['int7'][num_comparts*6:num_comparts*7]
#Rt8=numbers['int8'][num_comparts*6:num_comparts*7]
#Rt9=numbers['int9'][num_comparts*6:num_comparts*7]
#Rt10=numbers['int10'][num_comparts*6:num_comparts*7]
#Rt11=numbers['int11'][num_comparts*6:num_comparts*7]
#Rt12=numbers['int12'][num_comparts*6:num_comparts*7]
#Rt13=numbers['int13'][num_comparts*6:num_comparts*7]
Rt1 =np.sum(Rt1,axis=0)
Rt2 =np.sum(Rt2,axis=0)
Rt3 =np.sum(Rt3,axis=0)
Rt4 =np.sum(Rt4,axis=0)
Rt5 =np.sum(Rt5,axis=0)
#Rt6 =np.sum(Rt6,axis=0)
#Rt7 =np.sum(Rt7,axis=0)
#Rt8 =np.sum(Rt8,axis=0)
#Rt9 =np.sum(Rt9,axis=0)
#Rt10 =np.sum(Rt10,axis=0)
#Rt11 =np.sum(Rt11,axis=0)
#Rt12 =np.sum(Rt12,axis=0)
#Rt13 =np.sum(Rt13,axis=0)
Rt = np.append(Rt1[0:45],Rt2)
Rt = np.append(Rt[0:-1],Rt3)
Rt = np.append(Rt[0:-1],Rt4)
Rt = np.append(Rt[0:-1],Rt5)
#Rt = np.append(Rt[0:-1],Rt6)
#Rt = np.append(Rt[0:-1],Rt7)
#Rt = np.append(Rt[0:-1],Rt8)
#Rt = np.append(Rt[0:-1],Rt9)
#Rt = np.append(Rt[0:-1],Rt10)
#Rt = np.append(Rt[0:-1],Rt11)
#Rt = np.append(Rt[0:-1],Rt12)
#Rt = np.append(Rt[0:-1],Rt13)

In [26]:
#rnumbers = numbers.copy()

In [27]:
rnumbers = pickle.load( open( "data/numberscaserateincluded_post.p", "rb" ) )

In [28]:
rS1=rnumbers['int1'][num_comparts*0:num_comparts*1]
rS2=rnumbers['int2'][num_comparts*0:num_comparts*1]
rS3=rnumbers['int3'][num_comparts*0:num_comparts*1]
rS4=rnumbers['int4'][num_comparts*0:num_comparts*1]
rS5=rnumbers['int5'][num_comparts*0:num_comparts*1]

rS1 =np.sum(rS1,axis=0)
rS2 =np.sum(rS2,axis=0)
rS3 =np.sum(rS3,axis=0)
rS4 =np.sum(rS4,axis=0)
rS5 =np.sum(rS5,axis=0)

rS = np.append(rS1[0:45],rS2)
rS = np.append(rS[0:-1],rS3)
rS = np.append(rS[0:-1],rS4)
rS = np.append(rS[0:-1],rS5)


rR1=rnumbers['int1'][num_comparts*11:num_comparts*12]
rR2=rnumbers['int2'][num_comparts*11:num_comparts*12]
rR3=rnumbers['int3'][num_comparts*11:num_comparts*12]
rR4=rnumbers['int4'][num_comparts*11:num_comparts*12]
rR5=rnumbers['int5'][num_comparts*11:num_comparts*12]

rR1 =np.sum(rR1,axis=0)
rR2 =np.sum(rR2,axis=0)
rR3 =np.sum(rR3,axis=0)
rR4 =np.sum(rR4,axis=0)
rR5 =np.sum(rR5,axis=0)

rR = np.append(rR1[0:45],rR2)
rR = np.append(rR[0:-1],rR3)
rR = np.append(rR[0:-1],rR4)
rR = np.append(rR[0:-1],rR5)



rH1=rnumbers['int1'][num_comparts*5:num_comparts*6]
rH2=rnumbers['int2'][num_comparts*5:num_comparts*6]
rH3=rnumbers['int3'][num_comparts*5:num_comparts*6]
rH4=rnumbers['int4'][num_comparts*5:num_comparts*6]
rH5=rnumbers['int5'][num_comparts*5:num_comparts*6]

rH1 =np.sum(rH1,axis=0)
rH2 =np.sum(rH2,axis=0)
rH3 =np.sum(rH3,axis=0)
rH4 =np.sum(rH4,axis=0)
rH5 =np.sum(rH5,axis=0)

rH = np.append(rH1[0:45],rH2)
rH = np.append(rH[0:-1],rH3)
rH = np.append(rH[0:-1],rH4)
rH = np.append(rH[0:-1],rH5)


rUR1=rnumbers['int1'][num_comparts*7:num_comparts*8]
rUR2=rnumbers['int2'][num_comparts*7:num_comparts*8]
rUR3=rnumbers['int3'][num_comparts*7:num_comparts*8]
rUR4=rnumbers['int4'][num_comparts*7:num_comparts*8]
rUR5=rnumbers['int5'][num_comparts*7:num_comparts*8]

rUR1 =np.sum(rUR1,axis=0)
rUR2 =np.sum(rUR2,axis=0)
rUR3 =np.sum(rUR3,axis=0)
rUR4 =np.sum(rUR4,axis=0)
rUR5 =np.sum(rUR5,axis=0)

rUR = np.append(rUR1[0:45],rUR2)
rUR = np.append(rUR[0:-1],rUR3)
rUR = np.append(rUR[0:-1],rUR4)
rUR = np.append(rUR[0:-1],rUR5)


rD1=rnumbers['int1'][num_comparts*8:num_comparts*9]
rD2=rnumbers['int2'][num_comparts*8:num_comparts*9]
rD3=rnumbers['int3'][num_comparts*8:num_comparts*9]
rD4=rnumbers['int4'][num_comparts*8:num_comparts*9]
rD5=rnumbers['int5'][num_comparts*8:num_comparts*9]

rD1 =np.sum(rD1,axis=0)
rD2 =np.sum(rD2,axis=0)
rD3 =np.sum(rD3,axis=0)
rD4 =np.sum(rD4,axis=0)
rD5 =np.sum(rD5,axis=0)

rD = np.append(rD1[0:45],rD2)
rD = np.append(rD[0:-1],rD3)
rD = np.append(rD[0:-1],rD4)
rD = np.append(rD[0:-1],rD5)


rC1=rnumbers['int1'][num_comparts*4:num_comparts*5]
rC2=rnumbers['int2'][num_comparts*4:num_comparts*5]
rC3=rnumbers['int3'][num_comparts*4:num_comparts*5]
rC4=rnumbers['int4'][num_comparts*4:num_comparts*5]
rC5=rnumbers['int5'][num_comparts*4:num_comparts*5]

rC1 =np.sum(rC1,axis=0)
rC2 =np.sum(rC2,axis=0)
rC3 =np.sum(rC3,axis=0)
rC4 =np.sum(rC4,axis=0)
rC5 =np.sum(rC5,axis=0)

rC = np.append(rC1[0:45],rC2)
rC = np.append(rC[0:-1],rC3)
rC = np.append(rC[0:-1],rC4)
rC = np.append(rC[0:-1],rC5)


rU1=rnumbers['int1'][num_comparts*2:num_comparts*3]
rU2=rnumbers['int2'][num_comparts*2:num_comparts*3]
rU3=rnumbers['int3'][num_comparts*2:num_comparts*3]
rU4=rnumbers['int4'][num_comparts*2:num_comparts*3]
rU5=rnumbers['int5'][num_comparts*2:num_comparts*3]

rU1 =np.sum(rU1,axis=0)
rU2 =np.sum(rU2,axis=0)
rU3 =np.sum(rU3,axis=0)
rU4 =np.sum(rU4,axis=0)
rU5 =np.sum(rU5,axis=0)

rU = np.append(rU1[0:45],rU2)
rU = np.append(rU[0:-1],rU3)
rU = np.append(rU[0:-1],rU4)
rU = np.append(rU[0:-1],rU5)


rE1=rnumbers['int1'][num_comparts*1:num_comparts*2]
rE2=rnumbers['int2'][num_comparts*1:num_comparts*2]
rE3=rnumbers['int3'][num_comparts*1:num_comparts*2]
rE4=rnumbers['int4'][num_comparts*1:num_comparts*2]
rE5=rnumbers['int5'][num_comparts*1:num_comparts*2]

rE1 =np.sum(rE1,axis=0)
rE2 =np.sum(rE2,axis=0)
rE3 =np.sum(rE3,axis=0)
rE4 =np.sum(rE4,axis=0)
rE5 =np.sum(rE5,axis=0)

rE = np.append(rE1[0:45],rE2)
rE = np.append(rE[0:-1],rE3)
rE = np.append(rE[0:-1],rE4)
rE = np.append(rE[0:-1],rE5)

rI1=rnumbers['int1'][num_comparts*3:num_comparts*4]
rI2=rnumbers['int2'][num_comparts*3:num_comparts*4]
rI3=rnumbers['int3'][num_comparts*3:num_comparts*4]
rI4=rnumbers['int4'][num_comparts*3:num_comparts*4]
rI5=rnumbers['int5'][num_comparts*3:num_comparts*4]

rI1 =np.sum(rI1,axis=0)
rI2 =np.sum(rI2,axis=0)
rI3 =np.sum(rI3,axis=0)
rI4 =np.sum(rI4,axis=0)
rI5 =np.sum(rI5,axis=0)

rI = np.append(rI1[0:45],rI2)
rI = np.append(rI[0:-1],rI3)
rI = np.append(rI[0:-1],rI4)
rI = np.append(rI[0:-1],rI5)


rRt1=rnumbers['int1'][num_comparts*6:num_comparts*7]
rRt2=rnumbers['int2'][num_comparts*6:num_comparts*7]
rRt3=rnumbers['int3'][num_comparts*6:num_comparts*7]
rRt4=rnumbers['int4'][num_comparts*6:num_comparts*7]
rRt5=rnumbers['int5'][num_comparts*6:num_comparts*7]

rRt1 =np.sum(rRt1,axis=0)
rRt2 =np.sum(rRt2,axis=0)
rRt3 =np.sum(rRt3,axis=0)
rRt4 =np.sum(rRt4,axis=0)
rRt5 =np.sum(rRt5,axis=0)

rRt = np.append(rRt1[0:45],rRt2)
rRt = np.append(rRt[0:-1],rRt3)
rRt = np.append(rRt[0:-1],rRt4)
rRt = np.append(rRt[0:-1],rRt5)


In [29]:
T2=301
d = np.genfromtxt ('data/death.csv', delimiter=",")
c = np.genfromtxt ('data/cases.csv', delimiter=",")
vv= np.genfromtxt ('data/vac.csv', delimiter=",")
h = np.genfromtxt ('data/hospitalization_data.csv',delimiter=",")

#h= np.array([i for i in h[0]])
actual_case = np.zeros((T2,num_comparts))
actual_death = np.zeros((T2,num_comparts))
case_daily = np.zeros((T2,num_comparts))
actual_v = np.zeros((T2,num_comparts))
actual_hos = np.zeros((T2,num_comparts))
actual_hos2 = np.zeros((T2,num_comparts))
count = 0 
for i in range(0,T2):
    actual_case[i,:] = np.round(c[i,:],0)
    case_daily[i,:] = np.round(c[i,:],0)
    actual_death[i,:] = np.round(d[i,:],0)
    actual_hos[i] = np.round(h[i,:],0)
    if i>=300:   
        actual_v[i,:] = np.round(vv[i-300,:],0)
for i in range(0,T2):
    if i>0:
        actual_case[i,:] += actual_case[i-1,:]
        actual_death[i,:] += actual_death[i-1,:]
actual_case = actual_case[0:T2]
actual_death = actual_death[0:T2]
case_daily = case_daily[0:T2]
w_case_daily = case_daily.copy()
for i in range(case_daily.shape[1]):
    for t in range(6,case_daily.shape[0]):
        w_case_daily[t,i] = np.mean(case_daily[t-6:t+1,i])
actual_hos = actual_hos[0:T2]
actual_v = actual_v[0:T2]
t = np.linspace(0, T2-1, T2)

In [30]:
N =I + Rt +UR+ H + U + E +S+Vt+D

In [31]:
rC5[-1]-C[-1]

402612.4242601552

In [32]:
rC5[-1]

625737.1526857209

In [33]:
60000/430880

0.13924990716672855

In [34]:
Vt[121:150]

array([  99970.30512632,  198270.56111584,  294934.14476524,
        389993.77718442,  483481.52338964,  575428.79436479,
        665866.35119284,  754824.31078304,  842332.15276488,
        928418.72719552, 1013112.26281251, 1096440.37564031,
       1178430.07782276, 1259107.78659908, 1338499.3333855 ,
       1416644.4411218 , 1493583.11177328, 1569340.86911288,
       1643941.36423087, 1717406.77293023, 1789758.09169575,
       1861015.35899592, 1931197.82156687, 2000324.06008723,
       2068412.08482674, 2135479.40902522, 2201543.10570192,
       2266619.85208588, 2330725.96475206])

In [35]:
(E[121:150]+I[121:150]+U[121:150]+H[121:150]+D[121:150])/N[121:150]

array([0.01187076, 0.01203343, 0.01218202, 0.01231617, 0.01243553,
       0.01253982, 0.01262882, 0.01270234, 0.0127603 , 0.01280264,
       0.01282941, 0.01284069, 0.01283664, 0.01281748, 0.01278348,
       0.01244899, 0.01207202, 0.01166751, 0.01124648, 0.01081705,
       0.0103852 , 0.0099554 , 0.00953094, 0.00911424, 0.00870711,
       0.00831086, 0.00792644, 0.00755452, 0.00719554])

In [36]:
#rrnumbers = pickle.load( open( "numberscaserateincluded_post.p", "rb" ) )